## PyTorch Implementation

### Simple version

In [1]:
import torch
import numpy as np

x = np.array([0, 1, 2, 3, 4]).astype('float32').reshape(-1,1) 
y = x * 2 + 1

class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.layer = torch.nn.Linear(1,1)
    def forward(self, x):
        return self.layer(x)
model = Model()
Optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1000):
    x_tr = torch.from_numpy(x[:2,:1]).type(torch.FloatTensor)
    y_tr = torch.from_numpy(y[:2,:1]).type(torch.FloatTensor)
    y_pr = model(x_tr)
    loss = torch.pow(torch.abs(y_tr - y_pr),2)
    Optimizer.zero_grad()
    torch.sum(loss).backward()
    Optimizer.step()
print(model(torch.from_numpy(x).type(torch.FloatTensor)).detach().numpy())

[[1.0062771]
 [2.9965625]
 [4.986848 ]
 [6.9771333]
 [8.967419 ]]


### Detail version with monitoring variables

In [2]:
import torch
import numpy as np

x = np.array([0, 1, 2, 3, 4]).astype('float32').reshape(-1,1) 
y = x * 2 + 1

class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.layer = torch.nn.Linear(1,1)
    def forward(self, x):
        return self.layer(x)
model = Model()
Optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print('w=', list(model.parameters())[0].detach().numpy())
print('b=', list(model.parameters())[1].detach().numpy())
print()

for epoch in range(1000):
    x_tr = torch.from_numpy(x[:2,:1]).type(torch.FloatTensor)
    y_tr = torch.from_numpy(y[:2,:1]).type(torch.FloatTensor)
    y_pr = model(x_tr)
    loss = torch.pow(torch.abs(y_tr - y_pr),2)
    if epoch < 3:
        print(f'Epoch:{epoch}')
        print('y_pr:', y_pr.detach().numpy())
        print('y_tr:', y[:2,:1])
        print('loss:', loss.detach().numpy())
        print()
    Optimizer.zero_grad()
    torch.sum(loss).backward()
    Optimizer.step()
print(model(torch.from_numpy(x).type(torch.FloatTensor)).detach().numpy())

w= [[0.93893766]]
b= [0.509961]

Epoch:0
y_pr: [[0.509961 ]
 [1.4488987]]
y_tr: [[1.]
 [3.]]
loss: [[0.24013822]
 [2.4059153 ]]

Epoch:1
y_pr: [[0.519961 ]
 [1.4688987]]
y_tr: [[1.]
 [3.]]
loss: [[0.23043744]
 [2.3442714 ]]

Epoch:2
y_pr: [[0.5299569]
 [1.4888909]]
y_tr: [[1.]
 [3.]]
loss: [[0.22094053]
 [2.2834508 ]]

[[1.000002 ]
 [2.999999 ]
 [4.999996 ]
 [6.9999933]
 [8.99999  ]]
